> ⚠️ **Important:** Before running this notebook, make sure you have run the **Prejoining Instructions** section to install necessary packages.

## 🛠️ PREJOINING INTRUCTIONS 🛠️

### Prerequisite 

This workshop is suitable for those beginner to intermediate in R. It requires you know how to set your working directory and how to read data into R.

Using open source police recorded crime statistics this workshop will demonstrate how to map crime data in R using sf and ggplot. More specifically looking at the area of Surrey we will 

  1) briefly explore the crime data and introduce key topics in spatial data
  2) demonstrate how to join crime data to shapefiles and how to map data 
  3) identify how to map and calculate crime rate 

The datasets needed in this workshop include crime data, population statistics and shapefiles. Information on how to download these will be available in the R file names *downloading the data* but feel free to obtain these via git *https://github.com/UKDataServiceOpen/Mapping_Crime_Data_R_2025*. Ensure these are loaded into your environment before starting the workshop


### Set your working directory 

The working directory is just a file path on your computer that sets the default location of any files you read into R, or save out of R. You need to set your current working directory in order to follow along with this workshop, You can read more about working directories here  *https://bookdown.org/ndphillips/YaRrr/the-working-directory.html* 

You can use the *setwd()* function to set your current working directory, and the *getwd()* function to print your current working directory. 

I would suggest however, creating an R project with a version control repository. This will allow you to work with the complete set of code provided. To do so follow these steps from the task menu found at the top left of your RStudio;

File -> new project -> version control -> click git -> paste the github link in the ''repository url' box  *(https://github.com/UKDataServiceOpen/Mapping_Crime_Data_R_2025.git)* -> give your project directory a name -> tick open in new session -> click create new project.

Perfect! Now you can work along side me as I go through the various markdown files. 


### Install and load the packages required for this workshop

Below are a list of packages that need to be installed and loaded into your version of RStudio. To install a package use the function *install.packages("...")* - the quotation marks around the package names are essential. 

Once installed you then need to load the packages which can be down via the *library(...)* function - the quotation marks here are not essential. 


In [ ]:
knitr::opts_chunk$set(echo = TRUE)

## Install packages 

# install.packages("dplyr")
# install.packages("tidyr")
# install.packages("readr")
# install.packages("tibble")
# install.packages("janitor")
# install.packages("sf")
# install.packages("ggmap")
# install.packages("ggplot2")
# install.packages("ggspatial")
# install.packages("spdep")
# install.packages("leaflet")
# install.packages("RColorBrewer")
# install.packages("tmap")
# install.packages("cartogram")
# install.packages("prettymapr")
# install.packages("geojsonio") needed to convert spatial objects in colab for accurate visualisation

install.packages(c("dplyr", "tidyr", "readr", "tibble", "janitor", "sf",
                   "ggmap", "ggplot2", "ggspatial", "spdep", "leaflet",
                   "RColorBrewer", "tmap", "cartogram", "prettymapr", "geojsonio"))


In [ ]:
## Load packages

# for data reading/manipulation 
library(dplyr)
library(tidyr)
library(readr)
library(tibble)
library(janitor)
library(readxl)
# for spatial data and gis
library(sf)
library(ggmap)
library(ggplot2)
library(ggspatial)
library(spdep)
library(leaflet) 
library(RColorBrewer)
library(tmap)
library(cartogram)
library(prettymapr)
library(geojsonio)


### Connecting GitHub and R (extra step)

In order to clone a repository from Github you might be asked to set up a PAT (personal authentication token), if so follow these simple instructions from this link. *https://happygitwithr.com/https-pat.html*. (Do not share this token with anyone, it is a unique password for your Github account)

In summary: 


In [ ]:
usethis::create_github_token()

gitcreds::gitcreds_set()
#and then enter your token 


--------------------------------------------------------------------------------------------------------------------------


## TOPIC 1 - 🏙️ EXPLORING CRIME DATA 🏙️


In this first section I will show you how to read in the data, run some basic exploratory analysis and produce some point maps. 

### Install and Load packages

As always the first step is to load the necessary R packages via the library function. If you do not have these packages installed then please follow the instructions in the *Prejoining_Instructions.Rmd* file. 


In [ ]:
#install.packages(")
#install.packages(c("dplyr", "tidyr", "readr", "tibble", "janitor", "sf", 
                  # "ggmap", "ggplot2", "ggspatial", "spdep", "leaflet", 
                  # "RColorBrewer", "tmap", "cartogram", "prettymapr"))


In [ ]:
# for data reading/manipulation 
library(dplyr)
library(tidyr)
library(readr)
library(tibble)
library(janitor)
library(readxl)
# for spatial data and gis
library(sf)
library(ggmap)
library(ggplot2)
library(ggspatial)
library(spdep)
library(leaflet) 
library(RColorBrewer)
library(tmap)
#library(rgdal) this package is deprecated
library(prettymapr)
library(geojsonio)


  
### Downloading the crime data 

 We will be using crime data from *https://data.police.uk/*. This is the site for open data about policing and crime in England, Wales and Northern Ireland. You can download street-level crime, outcome, and stop and search data in clear and simple CSV format and explore the API containing detailed crime data and information about individual police forces and neighborhood teams. You can also download data on police activity, and a range of data collected under the police annual data requirement (ADR) including arrests and 101 call handling.
 
 We will be using data from 2023-2024 and in order to save time I have included the data in this R  Project under the Data folder. However, if you were interested in how I collected this then you can view the *downloading the data" doc* word document, In summary this would involve....
 
Select Downloads -> Select December 2023 to December 2024 -> select Surrey and click 'Include Crime Data'. Download and unzip the data into your working directory.

Read in just the month of February 2024. 


In [ ]:
#unzip(file.choose())

library(readr)
url <- "https://raw.githubusercontent.com/UKDataServiceOpen/Mapping_Crime_Data_R_2025/main/Data/crime_data/2024-02/2024-02-surrey-street.csv"
crime <- read_csv(url) %>% janitor::clean_names()


WE can use the head() and glimpse() function to explore our data.



In [ ]:
#explore variables
head(crime)
glimpse(crime)


Points, lines and polygon 

- Our coordinate variables (the latitude and longitude) are known as point data 
- The 'location' variable represents the line. This is normally define by a street or junction 
- The 'lsoa name' represent our polygon (borough, wards, districts etc). LSOA refers to the Lower Layer Super Output Areas which are a unit measure in census geography 


In this data, the "crime_type' column contains the general names of each of the different crimes.

Using the unique() function lists creates an array of the different possible values in the column.


In [ ]:
unique(crime$crime_type)



Before moving on to some of the more complicated spatial topics, lets create some frequency tables for each different crime_type.  

The table() can be used to create a frequency table for each different Primary.Type of crime.
By default, the table is sorted by the category. order() can be used to order the table by count. In this example, we list the top ten crimes by activity.


In [ ]:
counts = table(crime$crime_type)

counts = counts[order(counts, decreasing=T)]

print(counts[1:10])


Now we have a nice summarised table of all our crime counts. Lets go ahead and plot this to view this a bit better 



In [ ]:
par(mar = c(5,10,1,1))

y = barplot(counts[1:10], horiz=T, las=1, cex.names=0.7, col="whitesmoke")
 
text(1000, y, counts[1:10], pos=4, cex=0.8)


We can also improve these summary tables by using use the 'dplyr' for better readability and functionality. 



In [ ]:
#Table 

crime_summary <- crime %>%
  count(crime_type, sort = TRUE) %>%
  top_n(10)

print(crime_summary)

#Plot 

ggplot(crime_summary, aes(x = reorder(crime_type, n), y = n, fill = crime_type)) +
  geom_col(show.legend = FALSE) +
  coord_flip() +
  labs(title = "Top 10 Most Common Crimes in Surrey February 2024",
       x = "Crime Type", y = "Count") +
  theme_minimal()


### Simple Features and Projection methods

Simple Features is a common R language, also known as sf, that allow you to handle and manipulate the UoA (points, lines and polyons). Simple Features allows you store spatial objects.

Features refers to the property that linestring and polygons are built from points by straight line segments. 

One of the fastest growing packages in this area is [sf](https://github.com/r-spatial/sf), which gives you access to a whole host of features and functions for use with spatial data, including visualisation. html) to spatial data out there. For this exercise, we'll keep things simple, and focus on how to use sf to make spatial data visualisations in combination with ggplot. Should you want to know more, or would like additional resources on using spatial data in R, please do not hesitate to ask!


CRS and Projection: 

CRS is a coordinate-based local, regional or global system used to locate geographical entities. A spatial reference system defines a specific map projection, as well as transformations between different spatial reference systems. Spatial reference systems can be referred to using a SRID integer, including EPSG codes.

In short "Projection methods allow us to move move from the 3D to the 2D, CRS allow us to identify specific locations within these.

There are thousands of CRS, the most common being BNG and the WGS 84 

Each crs has an ESPG identifier
i.e. the BNG = 27700 (British National Grid)
i.e. the WGS 84 is 4326 (World Geodetic System)
i.e. the ETRS 1980 = 3035 (European Terrestial Reference System)


First step is to transform you ordinary data into an sf object using 'st_as_sf' - which converts our latitude and longitude to a geometry attribute

To recap, sf objects are just data-frames that are collections of spatial objects. Each row is a spatial object (e.g. a polgyon), that may have data associated with it (e.g. its area) and a special geo variable that contains the coordinates


In [ ]:
st_crs(crime)   # to check the crs

sf <- st_as_sf(crime,                                
                      coords = c("longitude", "latitude"),
                      crs = 4326,     
                      na.fail = FALSE)
st_crs(sf)

glimpse(sf)
head(sf)


Other functions 

- agr (atribute-geometry-relationship) = character vector. 
- Specifies for each non-geometry attribute column how it relates to the geometry, and can have one of following values: "constant", "aggregate", "identity". "constant" is used for attributes that are constant throughout the geometry (e.g. land use), "aggregate" where the attribute is an aggregate value over the geometry (e.g. population density or population count), "identity" when the attributes uniquely identifies the geometry of particular "thing", such as a building ID or a city name. The default value, NA_agr_, implies we don't know.


### Mapping point data 

Now we have an sf object which contains point-level, spatially sensitive data about Crime in Surrey 2019, We can now create a basic point map of these

#### Plot the point data


In [ ]:
ggplot() + 
  geom_sf(data = sf)


#### Colour the different crime type



In [ ]:
ggplot() + 
  geom_sf(data = sf, aes(col = crime_type))

#### with titles
ggplot() + 
  geom_sf(data = sf, aes(fill = crime_type, col = crime_type)) + 
  labs(title = "Crime Count in Surrey", 
       subtitle = "February 2024", 
       caption = "Police Recorded Crime Statistics")

# Go one step further and change theme and transparency
ggplot() + 
  geom_sf(data = sf, aes(color = crime_type), alpha = 0.6) +
  theme_minimal() +
  labs(title = "Crime Locations in Surrey",
       subtitle = "February 2024",
       caption = "Source: Police.uk")


#### Reference map / base map  



In [ ]:
ggplot() + 
  annotation_map_tile() +
  geom_sf(data = sf, aes(col = crime_type))


#### Sub-setting for just ASB 



In [ ]:
asb <- subset(sf, crime_type == "Anti-social behaviour") %>% 
  select(-c(1, 9, 10))
head(asb)


ggplot() +
  annotation_map_tile() +
  geom_sf(data = asb)


## Activity 1

How does this compare to the crime_type 'drugs'?

Steps; 
*1. Subset the data for the those crime types recorded as 'drugs', 
*2. create this into a new object like we did for ASB and name it 'drugs' 
*3. Using ggplot plot the point data over a base map (reference map)


In [ ]:
#1)
subset(sf, ..... ==  .....  )


#2) 
drugs <- subset(sf, ..... == ..... ) 


#3)
ggplot() +
  ..............() +
  geom_sf(data = .....) 


--------------------------------------------------------------------------------------------------------------------------

## TOPIC 2 - 🗺️ SHAPEFIlES 🗺️  


In this section we will be working with shapefiles. More specifically how to read in a shapefile and join this to our aggregated crime count data frame. From there we introduce classification methods as a way to better visualize crime counts. 

### Load packages

As always the first step is to load the necessary R packages via the library function. If you do not have these packages installed then please follow the instructions in the *Preliminary Task.Rmd* file. 


In [ ]:
# for data reading/manipulation 
library(dplyr)
library(tidyr)
library(readr)
library(tibble)
library(janitor)
library(readxl)
# for spatial data and gis
library(sf)
library(ggmap)
library(ggplot2)
library(ggspatial)
library(spdep)
library(leaflet) 
library(RColorBrewer)
library(tmap)
library(geojsonio)


### What is a Shapefile? 

They represent a geospatial vector that is used for GIS software. Shapefiles store both geographic location and its associated attribute information 

The Shapefile format stores the data as primitive geometric shapes like points, lines, and polygons. These shapes, together with data attributes that are linked to each shape, create the representation of the geographic data.

They contain four mandatory file extensions (.shx, .shp, .dbf and the .prj). 
- The .shp contains the geometry data (a 2D axis ordering of coordinate data)
- The .shx contains the positional index of the feature geometry 
- The .dbf contins the attributes for each shape
- The .prj contains the cs and projection information

In criminological research, the LSOA is quite frequently used as the main census geography 


### Where to obtain shapefiles 

I collected my shapefile data via the UKDS Census Support *(https://borders.ukdataservice.ac.uk/bds.html)*. If you want specific information about how to use the website, please refer to the *Downloading the data* document again 


### Read in the Shapefile for 'Surrey Heath' 


In [ ]:
# Define the base URL for the shapefile components
base_url <- "https://raw.githubusercontent.com/UKDataServiceOpen/Mapping_Crime_Data_R_2025/main/Data/shapefile_data/"

# List of necessary files for the shapefile
shp_files <- c("england_lsoa_2021.shp", "england_lsoa_2021.dbf", "england_lsoa_2021.prj", "england_lsoa_2021.shx")

# Create local directory for downloaded shapefile
dir.create("shapefile_data", showWarnings = FALSE)

# Download each shapefile component
for (file in shp_files) {
  download.file(paste0(base_url, file), paste0("shapefile_data/", file), mode = "wb")
}

# Read the shapefile
shp_file <- sf::st_read("shapefile_data/england_lsoa_2021.shp")

# Print summary
shp_file


You can also use the head() function in shapefiles!



In [ ]:
head(shp_file)
#or use 'View(shp_file) to view the full dataset which will open up in a new panel


*Further Information - To clarify, this is an 'empty shapefile', it simply contains the boundary profile of Surrey Heath and does not contain any further attribute information. However, if it did contain further attribute information such as the crime counts, population statistics, IMD counts, then you would not need to join the data as we do in this workshop, but instead you could layer the shapefile over our simple feature object created in Topic 1. More information on this method is available from the workshop that was held in February - all resources are available via the 'Feb_2021' folder from the github link [https://github.com/UKDataServiceOpen/Crime_Data_in_R.git] *

Lets plot the empty shapefile (without attribute data) for Surrey Heath to see what we're actually looking at.


In [ ]:
## Plot the Shapefile 
ggplot() + 
  geom_sf(data = shp_file)


As you can seem, we have a map that details the borders (i.e. shape) between each LSOA in Surrey. 

This workshop instead joins the 'crime' datatset (in tibble format) to the above shapefile. Our newly created object 'shp_file' is in fact a sf object, which is short for a 'simple feature object'. You can check this by typing *class(shp_file)*. 


In [ ]:
class(shp_file)



So in total, the shapefile consits of 5 variables. The first 4 variables indicate information about that specific LSOA, we are given the name, LSOA code and LSOA name. We can ignore the column 'label' as this is just another reference point.


The column I want to draw attention to is the 'geometry column' 


In [ ]:
attributes(shp_file$geometry)



The geometry column can be split into two key sections; the feature and the geometry 
  - The feature in this case is our polygon level (referenced by the multipolygon) which is in fact a  *simple feature geometery list- column (sfc)* 
  - The geometery are the numbers that follow, and more technically known as a *'simple feature geometry (sfg)*

The column in the sf data.frame that contains the geometries is a list, of class sfc. We can retrieve the geometry list-column in this case by using st_geometry. 


In [ ]:
st_geometry(shp_file)



Now we have a basic understanding of what a shapefile is and how we can import them into r, the next step is run some data manipulation and create some new dataframes that can work with the format of shapefiles. 



### Group the crimes per lsoa 

The original crime data set contains the individual count of reported crime types across LSOAS, therefore the LSOAs are repeated multiple times. This is because you would expect to see multiple crime counts in one LSOA.

In order to highlight how many crimes have occurred in each LSOA, we can count the crimes per LSOA and obtained grouped statistics.


In [ ]:
crimes_grouped_by_lsoa <- crime %>%
  group_by(lsoa_code) %>%
  summarise(count=n())

head(crimes_grouped_by_lsoa)


### Merge the shapefile to the crime dataset

In our new object you will see two variables, the LSOA and the count of crime in each one.  

We can now join the Shapefile (the geospatial vector) and the crimes_grouped_by_losa (the aggregated data)

To join the crimes per lsoa to the shapefile we can use the left_join function that returns all the rows of the table on the left side of the join and matching rows for the table on the right side of join.


In [ ]:
surrey_lsoa <- left_join(shp_file, crimes_grouped_by_lsoa, by = c("lsoa21cd" = "lsoa_code"))

head(surrey_lsoa)

st_geometry_type(surrey_lsoa)    #view the geometery type 
st_bbox(surrey_lsoa)             #obtains the objects value as specific units 


#The spatial extent of a shapefile or R spatial object represents the geographic “edge” or location that is the #furthest north, south east and west. Thus is represents the overall geographic coverage of the spatial object.


Now lets map the new data!



In [ ]:
#map the data
ggplot() + 
  annotation_map_tile() + 
  geom_sf(data = surrey_lsoa, aes(fill = count), alpha = 0.5) + 
  scale_fill_gradient2(name ="Number of crimes")

## imroved mapping, 
ggplot(surrey_lsoa) + 
  geom_sf(aes(fill = count), color = "black", lwd = 0.2) +
  scale_fill_viridis_c(option = "plasma", name = "Crime Count") + #better colour schemes
  labs(title = "Crime Counts by LSOA in Surrey",
       caption = "Source: Police.uk") +
  theme_minimal()


### Plotting via the 'tmap' package

The tmap package allows you tp create thematic maps, the syntax is very similar to the ggplot2. Each map can be plot as an image or as an interactive map via the *tmap_mode("view" / "plot" )* function. 


In [ ]:
tmap_mode("view")

tm_shape(surrey_lsoa) + 
  tm_fill("count") + 
  tm_borders("green", lwd = 0.7, alpha = 0.5)
  #tm_text("name", size = "AREA", col = "black")
  #tmap_style("col_blind")

## another example
tm_shape(surrey_lsoa) + 
  tm_polygons("count", palette = "Reds", title = "Crime Count") + 
  tm_borders("black", lwd = 0.5) +
  tm_layout(main.title = "Crime Counts by LSOA in Surrey")


### Classification methods

How can we better visualise counts? Count data does not equally represent the population distribution at hand, tmaps allows you to alter the characteristics of thematic maps via the 'styles' function. The different styles result in different binning techniques.  Now, when mapping quantitative data such as crime counts, typically the variables needed to be put into to 'bins'. As seen in the previous example, the default binning applied to highlight the LSOAs grouped started from 1-10, 11-20, 21-20, 31-40, 41-50 and 51-60 crimes.

These bins were decided on automatically, however we can define more accurate classes that best reflect the distributional character of the data set.


In this example I've used the "kmeans", "jenks" and "sd".

- k-means is a method of vector quantisation, originally from signal processing, that aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean, serving as a prototype of the cluster.
- Jenks (also known as natural breaks, or goodness of fit variance) classification aims to arrange a set of values into natural classes, that is the most optimal class range found naturally. This method minimizes the variation within each range, so the areas within each range are as close as possible in value to each other. 
- sd classification is a standarised measure of observations deviated from the mean. By showing which values are above or below the mean, this method helps to show which locations are above or below an average mean.


Here's a quick example to show why different classification methods matter


In [ ]:
hist(surrey_lsoa$count, breaks = 30, main = "Crime Count Distribution", col = "lightblue")



Lets look at some more detailed examples; 



In [ ]:
a <- tm_shape(surrey_lsoa) + 
  tm_fill("count", style = "kmeans") + 
  tm_borders(alpha = 0.3)

b <- tm_shape(surrey_lsoa) + 
  tm_fill("count", style = "jenks") + 
  tm_borders(alpha = 0.3)

c <- tm_shape(surrey_lsoa) + 
  tm_fill("count", style = "sd") + 
  tm_borders(alpha = 0.3)


## tmap_arrange

tmap_mode("plot")
tmap_arrange(a, b, c)


### Using categorical variables (tm_facets)

Just like the tmap_arrange function, tmap_facets are way to produce side-by-side maps (known as *small multiples*). It is similar to the 'facet_grid' function in ggplot2

Following the rdocumentation [https://www.rdocumentation.org/packages/tmap/versions/3.3-2/topics/tm_facets] "Small multiples can be created in two ways: 1) by specifying the by argument with one or two variable names, by which the data is grouped, 2) by specifying multiple variable names in any of the aesthetic argument of the layer functions (for instance, the argument col in tm_fill)." 

Typically tm_facets are defined by a categorical variables. For example in this example, I am using tm_facets() to seperate the map into multiple components by lsoa (This isnt't the best example of a categorical variable, but something like the urban or rural landscape, or deprivation decile, would be more of interest in criminology). You could for example download the 2011 rural/urban classification from open geography portal and join this to our 'surrey_lsoa' sf object (using left_join). 

If you are interested in doing so the dataset can be found here;
[https://www.ons.gov.uk/methodology/geography/geographicalproducts/ruralurbanclassifications/2011ruralurbanclassification]


In [ ]:
tm_shape(surrey_lsoa) +
  tm_fill("count",
          style = "quantile",
          palette = "Blues",
          thres.poly = 0) + 
  tm_facets(by="name", 
            free.coords=TRUE, 
            drop.shapes=TRUE) +
  tm_layout(legend.show = FALSE,
            title.position = c("center", "center"), 
            title.size = 20) +
  tm_borders(alpha = 0.5)


### Map Layouts - additional features of tmap (optional task)

#### map style 


In [ ]:
tm_shape(surrey_lsoa) + 
  tm_fill("count", style = "sd") + 
  tm_borders(alpha = 0.3) + 
  tmap_style("col_blind")


#### map legends



In [ ]:
tm_shape(surrey_lsoa)+
  tm_fill("count", 
          style = "quantile", 
          palette = "Blues", 
          legend.hist = TRUE, 
          legend.is.portrait = TRUE,
          legend.hist.z = 0.1) +
  tm_layout(legend.height = 0.45, 
            legend.width = 0.35,
            legend.outside = FALSE,
            legend.position = c("right", "bottom"),
            frame = FALSE) +
  tm_borders(alpha = 0.5)


#### compass, scale bar and grid



In [ ]:
tm_shape(surrey_lsoa)+
  tm_fill("count", 
          style = "quantile", 
          palette = "Blues", 
          legend.hist = TRUE, 
          legend.is.portrait = TRUE,
          legend.hist.z = 0.1) +
  tm_layout(legend.height = 0.45, 
            legend.width = 0.35,
            legend.outside = FALSE,
            legend.position = c("right", "bottom"),
            frame = FALSE) +
  tm_borders(alpha = 0.5) +
  tm_compass(type="8star", size = 2) +   #compass 
  tm_scale_bar(width = 0.15) +           #scale bar 
  tm_grid()                              #grid


## Activity 2

1. Explore some of the different classification methods such as "bclust" and "hclust"  - what are the main differences? To get help on the different methods available use *??tmap-package* or search in the help tab 

2. Assign your new bclust and hclust classification maps into separate objects (call them "h" and "b" and plot them together using tmap_arrange()


3. Plot an interactive map using the "bclust" classification method by changing the command in the tmap_mode() function 

   


In [ ]:
#1)

... <- tm_shape(.....) + 
  tm_fill("...", style = "....") + 
  tm_borders(alpha = 0.3)


... <- tm_shape(........) + 
  tm_fill("...", style = ".....") + 
  tm_borders(alpha = 0.3)


#2) 

tmap_arrange(...., ....)



#3) 

tmap_mode("....")

tm_shape(.....) + 
  tm_fill(....., style = .....) + 
  tm_borders(alpha = 0.3)


 


--------------------------------------------------------------------------------------------------------------------------

## TOPIC 3 - 🏠 CENSUS 🏠 

In this section we incorporate some census data onto our shapefile so we can calculate and explore the differences between crime rate and crime counts. We then introduce cartograms as an alternative thematic map. 

In theory showing that crime rate accounts for population differences and is a more meaningful measure than raw counts.

As always the first step is to load the necessary R packages via the library function. If you do not have these packages installed then please follow the instructions in the *Preliminary Task.Rmd* file. 

### Load Packages


In [ ]:
# for data reading/manipulation 
library(dplyr)
library(tidyr)
library(readr)
library(tibble)
library(janitor)
library(readxl)
# for spatial data and gis
library(sf)
library(ggmap)
library(ggplot2)
library(ggspatial)
library(ggspatial)
library(spdep)
library(leaflet) 
library(RColorBrewer)
library(tmap)
library(geojsonio)


### Read in the population statistics

Count data is not entirely accurate of population density. Whilst the code in Section_1 and Section_2 might help us identify interesting patterns, point-level open crime data is rarely used in isolation for detailed analysis. 

For one thing, the data are points are geomasked. This means that points are highly likely to be overlapped, giving a skewed picture of the distribution. There are ways round this, such as through jittering or applying census based data. To view more information about jittering please view the 'Additional Topic.rmd' file,


### Census 2022

We will be using the new census 2022 data to calculate the crime rate across both residential population vs working population,

The data was downloaded from CKAN which can be accessed from the front page of our website *https://statistics.ukdataservice.ac.uk/dataset/?vocab_Area_type=Lower%20Super%20Output%20Areas*

I simply searched for 'residents' and 'work' as I want to calculate the difference in crime counts compared to the usual resident population and then to the usual workday population. Download the super layer output areas and once opened in excel, you can select just the rows that correspond to 'Surrey Heath'.

But have no fear if you don't know how to do this or are simply feeling a little lazy (I get it), you can just read in the data that has been cleaned for you in the Data folder. There is more information via the *Dowloading the Data* doc if you are interested in the download process. 

So lets read in the data and use the clean_names() function again to lowercase and tidy all variable names. 

We are going to start with just the resident population which is the data set named 'res_count.xlsx'


In [ ]:
url <- "https://raw.githubusercontent.com/UKDataServiceOpen/Mapping_Crime_Data_R_2025/main/Data/census_data/resident_population.xlsx"

# Download the Excel file to a temporary location before reading
temp_file <- tempfile(fileext = ".xlsx")
download.file(url, temp_file, mode = "wb")

residential_count <- read_excel(temp_file) %>% janitor::clean_names() %>%
  slice(-c(1:7)) %>%
  slice(-c(56, 57)) %>% 
  rename(lsoa = x2, #rename the variables 
         name = ts001_number_of_usual_residents_in_households_and_communal_establishments,
         res_count = x3) 

head(residential_count)


### Join the data to our new shapefile

The next step is to join these new statistics to our previously created shapefile named 'surrey_lsoa". We do thus by using the left_join() function in the dplyr package and attach them by the same LSOA codes. 


In [ ]:
surrey_lsoa <- left_join(surrey_lsoa, residential_count, by = c("lsoa21cd"="lsoa"))

head(surrey_lsoa)


Now you will see the census data has merged into the shapefile, Great lets move on to calculating crime rate.


### How to calculate the crime rate?

A crime rate is calculated by dividing the number of reported crimes by the total population, and then multiplying by 100,000. 

So for our dataset, we take the count variable, divide by the 'pop' variable (workday or residential), and then times by 1000 (in this instance we use 1000 as this is the average population of an LSOA, if you were using larger UoA you can choose to multiply by 100,000. Just remember what affect this will have on your rate and how this then interprets across your results.

In order to work out the crime rate, we need to create a new variable that takes the count/pop*1000. We can use the mutate() function from the dplyr package to calculate this for us. 


In [ ]:
surrey_lsoa <- surrey_lsoa %>% 
  mutate(crime_rate = (count/res_count*1000))
         
head(surrey_lsoa)


### Now lets explore these trends using ggplot and tmaps;

#### First ggplot


In [ ]:
ggplot() + 
  annotation_map_tile() + 
  geom_sf(data = surrey_lsoa, aes(fill = crime_rate), alpha = 0.5) + 
  scale_fill_gradient2(name ="Crime Rate")

##Improve Mapping
ggplot(surrey_lsoa) + 
  geom_sf(aes(fill = crime_rate), color = "black", lwd = 0.2) +
  scale_fill_viridis_c(option = "magma", name = "Crime Rate per 10,000") +
  labs(title = "Crime Rate by LSOA in Surrey",
       caption = "Source: Police.uk & Census 2022") +
  theme_minimal()


#### What about tmaps 



In [ ]:
tm_shape(surrey_lsoa) + 
  tm_fill("crime_rate", style = "quantile") + 
  tm_borders(alpha = 0.3)

##Improved Mapping 
tmap_mode("plot")

tm_shape(surrey_lsoa) + 
  tm_polygons("crime_rate", style = "jenks", palette = "Reds", 
              title = "Crime Rate (per 10,000)") + 
  tm_borders("black", lwd = 0.5) +
  tm_layout(main.title = "Crime Rate by LSOA in Surrey")


### Cartograms and ggplot

A cartogram is a type of map where different geographic areas are modified based on a variable associated to those areas. There are two two types of cartograms: contiguous vs non-contiguous (sharing a common border). 

The cartogram package allows to build cartograms in R. It requires a geospatial object as input, with a numeric variable in the data slot. This variable will be used to distort region shape.


In [ ]:
# library(cartogram)
# 
# #In our data set we have a variable “res_count_wrk” which refers to the total number of people in our LSOA
# cart <- cartogram_cont(surrey_lsoa, weight = "res_count")
# 
# 
# ## simple plot
# ggplot(cart) +
#   geom_sf()
# 
# 
# ## fill with our count variable
# ggplot(cart) +
#   geom_sf(aes(fill = pop_count_wrk))
# 
# 
# ## add in some aesthetics
# ggplot(cart) +
#   geom_sf(aes(fill = pop_count_wrk),
#           color = "gray50",
#           linetype = 1,
#           lwd = 0.35) +
#   scale_fill_gradientn(colours = heat.colors(n =10,
#                                             alpha = 0.5,
#                                             rev = TRUE)) +
#   theme_gray() +
#   labs(title = "Surrey Heath: Population by LSOA",
#        subtitle = "August 2020")
# ```


## Activity 3

We have successfully mapped the residential population, now lets do the same with the variable workday (economic activity). First step is to read and clean the data as seen here:


In [ ]:
url <- "https://raw.githubusercontent.com/UKDataServiceOpen/Mapping_Crime_Data_R_2025/main/Data/census_data/workday_population.xlsx"

# Download the Excel file to a temporary location before reading
temp_file <- tempfile(fileext = ".xlsx")
download.file(url, temp_file, mode = "wb")

workday_count <- read_excel(temp_file) %>% janitor::clean_names() %>%
  slice(-c(1:7)) %>%
  slice(-c(56, 57)) %>% 
 rename(lsoa = x2, #rename the variables 
         name = wd102ew_population_density, 
         work_count = x3) 

head(workday_count)
# head(residential_count) the previous code


In [ ]:
surrey_lsoa <- left_join(surrey_lsoa, workday_count, by = c("lsoa21cd"="lsoa"))

head(surrey_lsoa)


So now you've added the work population count, it time to calculate the crime rate. Follow the steps below to try and complete the activity. 


Steps: 

1  First calculate the crime rate 
2. Plot using ggplot 
3. Plot using tmap 
4. Plot both maps (residential and workday) together using tmap_arrange 
5. Plot a cartogram of residential population

Is there a difference between the crime rate when using workday population compared to residential population? Would we expect to see these trends?


In [ ]:
# 1) First calculate the crime rate, assign it a new variable and named it crimerate2

surrey_lsoa <- surrey_lsoa %>% 
  mutate(crime_rate2 = (count/......)*...)


#2) Plot using ggplot 

ggplot() + 
  annotation_map_tile() + 
  geom_sf(data = ....., aes(fill = ......), alpha = 0.5) + 
  scale_fill_gradient2(name ="Crime Rate")


#3) Plot using tmap 

tm_shape(surrey_lsoa) + 
  tm_fill(....., style = "quantile") + 
  tm_borders(alpha = 0.3)


#4) Compare the workday vs residential population 

e <- tm_shape(surrey_lsoa) + 
  tm_fill(....., style = "quantile", title = "Workday Pop") + 
  tm_borders(alpha = 0.3)

f <- tm_shape(surrey_lsoa) + 
  tm_fill(....., style = "quantile", title = "Residential pop") + 
  tm_borders(alpha = 0.3)


tmap_arrange(...., .....)



#5) Cartogram

# ggplot(.....) + 
#   geom_sf(aes(fill = ......), 
#           color = "gray50", 
#           linetype = 1, 
#           lwd = 0.35) + 
#   scale_fill_gradientn(colours = heat.colors(n =10, 
#                                             alpha = 0.5, 
#                                             rev = TRUE)) + 
#   theme_gray() + 
#   labs(title = "Surrey Heath: Population by LSOA", 
#        subtitle = "August 2020")


 


--------------------------------------------------------------------------------------------------------------------------

## TOPIC 4 - 🛰️ SPATIAL ANALYSIS 🛰️


In this final section I will demonstrate how to run spatial interpolation (which is the process pf using points with known values to estimate values at other unknown points). We will then move on to creating heat maps in both GGPLOT2 and Leaflet. La

### Load packages

As always the first step is to load the necessary R packages via the library function. If you do not have these packages installed then please follow the instructions in the *Preliminary Task.Rmd* file. 


In [ ]:
# for data reading/manipulation 
library(dplyr)
library(tidyr)
library(readr)
library(tibble)
library(janitor)
library(readxl)
# for spatial data and gis
library(sf)
library(ggmap)
library(ggplot2)
library(ggspatial)
library(spdep)
library(leaflet) 
library(RColorBrewer)
library(tmap)
library(geojsonio)

install.packages("zoo")
library(zoo) #for rolling averages


### Spatial Interpolation 

Spatial interpolation is a statistical technique used to predict or estimate values at unsampled locations within an area covered by existing observations. It's based on the principle that spatially or temporally close points tend to have similar values—a concept known as spatial autocorrelation. In the context of crime data, spatial interpolation can be particularly useful for several reasons:

*1. Fill Gaps: Estimate crime rates in areas where data are missing or not collected, providing a more complete spatial representation of crime.
*2. Smooth Crime Rates: Generate continuous surfaces of crime rates or risk across a geographic area, helping to identify hotspots and patterns that might not be evident from discrete data points alone.
*3. Resource Allocation: Inform law enforcement and public safety officials about potential areas of increased crime activity, aiding in strategic planning and resource allocation.
*4. Public Information: Provide communities with detailed maps of crime risk, enhancing awareness and preventive measures.

Lets explore our data. 


In [ ]:
# crime <- read_csv("/Users/user/Documents/Mapping_Crime_Data_R_2025/Data/crime_data/2024-02/2024-02-surrey-street.csv") %>% janitor::clean_names()
# 
# summary(crime)

url <- "https://raw.githubusercontent.com/UKDataServiceOpen/Mapping_Crime_Data_R_2025/main/Data/crime_data/2024-02/2024-02-surrey-street.csv"
crime <- read_csv(url) %>% janitor::clean_names()


We know that there are 6806 observations overall, but what about just for Surrey Heath which is a boundary area within the county of Surrey. 

We can use filter only those crimes that have taken place in Surrey Heath by using the dplyr and stringr packages


In [ ]:
library(stringr)
surrey_heath_crimes <- crime %>%
  filter(str_detect(lsoa_name, "^Surrey Heath"))

# Explore the filtered dataset
head(surrey_heath_crimes)
summary(surrey_heath_crimes)


This code filters the original crime dataset to keep only the records associated with LSOAs in the Surrey Heath area, based on the naming convention in the lsoa_name column, and stores this subset of data in a new variable for further analysis or operations.

We can now see that there are 464 observations, that is individual crime counts, across the boundry area of Surrey Heath. 

We know need to group these individual crime counts by lsoas in order to obtain the aggregated statistics. This has been repeated in multiple sections so I won't expand too much here. We will create this into a new object named 'cleaned_crimes_grouped_by_lsoas' 


In [ ]:
cleaned_crimes_grouped_by_lsoa <- surrey_heath_crimes %>%
  group_by(lsoa_code) %>%
  summarise(count = n())

# View the count of crimes by type
cleaned_crimes_grouped_by_lsoa


So what can we see? Well there are now 52 observations, in this case representing the number of LSOAs in Surrey Heath. However, if we compare this to our Shapefile of Surrey Heath, there are 55 observations which tells us there are three missing LSOAs. 

We can run a spatial interpolation to account for this by estimating the missing crime counts based on the average of neighboring LSOAs. 

Lets first identify the missing LSOA in the crime data set. We can use the setdiff() function from base R to do so...


In [ ]:
missing_lsoas <- setdiff(shp_file$lsoa21cd, cleaned_crimes_grouped_by_lsoa$lsoa_code)



There we go! We have identified the missing LSOA are "E01030810" "E01030790" "E01030771" 
In order to now run the spatial interpolation on the average of neighboring LSOAs, spatial visualization is required in order to understand if these LSOA does in fact have any neighboring LSOAs to establish a mean. This is because if the LSOA does not have any neighboring areas, we will not be able to establish a mean and therefore estimate a crime count value.

We can first establish a list of neighbors where each element corresponds to a LSOA within the shapefile (shp_file) and contains the indices of neighboring LSOAS.


In [ ]:
neighbors <- st_touches(shp_file, shp_file)



This allows to now identify the LSOAs with no neighbors. This line uses sapply() to apply a function to each element of the neighbors list. The function checks if the length of each element (list of neighbor indices) is 0, which would indicate that an LSOA has no neighbors.



In [ ]:
# Identify LSOAs with no neighbors
no_neighbors <- sapply(neighbors, function(x) length(x) == 0)


We then want to extract the indices (or IDs) of LSOAs with no neighbors and then extract the corresponding LSOAS. 



In [ ]:
# Extract the indices or IDs of LSOAs with no neighbors
lsoas_no_neighbors <- which(no_neighbors)

# Extract the corresponding LSOA codes or IDs
lsoas_with_no_neighbors <- shp_file$lsoa_code[lsoas_no_neighbors]


If lsoas_with_no_neighbors is empty, all LSOAs have at least one neighbor. If lsoas_with_no_neighbors contains LSOA codes or IDs, these are the LSOAs with no neighboring LSOAs within your dataset.

This would mean that we know that the LSOA does in fact have neighboring areas and therefore we can run some calculations on our neighboring areas to interpolate that missing value. For each missing LSOA, we need to find neighboring LSOAs and calculate the average crime count. 

This function iterates over a list of LSOAs (missing_lsoas) that lack crime data. For each missing LSOA, it:

*Identifies neighboring LSOAs that share a boundary with it.
*Gathers crime counts from these neighboring LSOAs.
*Calculates the average crime count from these neighbors.
*Rounds this average to the nearest whole number.
*Prints out this average for verification.
*If the average crime count is not calculated properly (e.g., it's NA or 0), it sets a default value of 1.
*Finally, it adds this calculated average crime count back into the dataset for the missing LSOA, effectively estimating the crime count for areas where it was originally missing

This process helps fill in gaps in the dataset, ensuring each LSOA has a crime count value, which is crucial for comprehensive spatial analysis.


In [ ]:
for(missing_lsoa in missing_lsoas) {
    neighbors_indices <- st_touches(shp_file[shp_file$lsoa21cd == missing_lsoa, , drop = FALSE], shp_file)[[1]]
    neighbor_lsoas <- shp_file$lsoa21cd[neighbors_indices]

    # Extracting crime counts for neighboring LSOAs
    neighbor_crime_counts <- cleaned_crimes_grouped_by_lsoa %>% 
                             filter(lsoa_code %in% neighbor_lsoas) %>%
                             pull(count) # Use pull to extract the 'count' column as a vector

    # Calculating the average crime count
    avg_crime_count <- mean(neighbor_crime_counts, na.rm = TRUE)
    
    # Round the average to the nearest whole number
    avg_crime_count_rounded <- round(avg_crime_count)

    # Debug: print the average crime count to verify
    print(paste("Average crime count for missing LSOA", missing_lsoa, ":", avg_crime_count))

    # Check if avg_crime_count is not calculated properly
    if(is.na(avg_crime_count) || avg_crime_count == 0) {
      # Apply a different strategy or set a default value other than 0 if that makes sense in your context
      avg_crime_count <- 1 # Example: setting a default value if mean calculation fails
    }

    # Update the dataset with the calculated average for the missing LSOA
    cleaned_crimes_grouped_by_lsoa <- rbind(cleaned_crimes_grouped_by_lsoa, data.frame(lsoa_code = missing_lsoa, count = avg_crime_count_rounded))
}


class(cleaned_crimes_grouped_by_lsoa)


Did you notice how the object 'cleaned_crimes_grouped_by_lsoas' has now changed from 52 observations to 55 observations. Lets see whats happened with that missing LSOA



In [ ]:
cleaned_crimes_grouped_by_lsoa[55, ]



Lets compare the previous surrey_lsoa (with the misssing lsoa data) and the surrey_lsoa_new (no missing lsoa data). Lets look at how these might be different both statistically and visually: 



In [ ]:
# Ensure both datasets are spatially joined to the original shapefile
surrey_lsoa <- left_join(shp_file, crimes_grouped_by_lsoa, by = c("lsoa21cd" = "lsoa_code"))
surrey_lsoa_new <- left_join(shp_file, cleaned_crimes_grouped_by_lsoa, by = c("lsoa21cd" = "lsoa_code"))


In [ ]:
# Statistically Compare 
mean_original <- mean(surrey_lsoa$count, na.rm = TRUE)
mean_imputed <- mean(surrey_lsoa_new$count)

median_original <- median(surrey_lsoa$count, na.rm = TRUE)
median_imputed <- median(surrey_lsoa_new$count)

# Print the results
cat("Mean Crime Count - Original Data (with missing):", mean_original, "\n")
cat("Mean Crime Count - Imputed Data (no missing):", mean_imputed, "\n\n")

cat("Median Crime Count - Original Data (with missing):", median_original, "\n")
cat("Median Crime Count - Imputed Data (no missing):", median_imputed, "\n")


In [ ]:
# Visually Compare
# Plotting the original dataset with missing value
ggplot() +
  geom_sf(data = surrey_lsoa, aes(fill = count), color = NA) +
  scale_fill_viridis_c(option = "plasma", na.value = "grey", guide = guide_legend(title = "Crime Count")) +
  ggtitle("Original Data with Missing LSOA Crime Count") +
  theme_minimal()

# Plotting the dataset with imputed value
ggplot() +
  geom_sf(data = surrey_lsoa_new, aes(fill = count), color = NA) +
  scale_fill_viridis_c(option = "plasma", guide = guide_legend(title = "Crime Count")) +
  ggtitle("Imputed Data with No Missing LSOA Crime Count") +
  theme_minimal()


## Heat Maps 

As I'm specifically interested in creating heatmaps of ASB across surrey heath, there is a little bit of data preparation needed. The steps are just repeated from above but they can be summarised as such:

*1) filter for asb within the crime dataframe (and within surrey heath, although this is not necessary as the shapefile is from surrey heath, but i want to impute some missing values before hand) 
*2) group by lsoa and summarise the counts
*3) find the missing lsoas
*4) impute the missing values for the lsoas
*5) left_join to the shapefile


In [ ]:
# Filter for ASB within Surry Heath
asb_data <- crime %>%
  filter(crime_type == "Anti-social behaviour") %>% 
  filter(str_detect(lsoa_name, "^Surrey Heath"))

# Group by LSOA and summarise the counts
asb_grouped_by_lsoa <- asb_data %>%
  group_by(lsoa_code) %>%
  summarise(count=n())

# Find the missing LSOAs
missing_lsoas <- setdiff(shp_file$lsoa21cd, asb_grouped_by_lsoa$lsoa_code)

# Impute the missing values 
for(missing_lsoa in missing_lsoas) {
    neighbors_indices <- st_touches(shp_file[shp_file$lsoa21cd == missing_lsoa, , drop = FALSE], shp_file)[[1]]
    neighbor_lsoas <- shp_file$lsoa21cd[neighbors_indices]

    # Extracting crime counts for neighboring LSOAs
    neighbor_crime_counts <- asb_grouped_by_lsoa %>% 
                             filter(lsoa_code %in% neighbor_lsoas) %>%
                             pull(count) # Use pull to extract the 'count' column as a vector

    # Calculating the average crime count
    avg_crime_count <- mean(neighbor_crime_counts, na.rm = TRUE)
    
    # Round the average to the nearest whole number
    avg_crime_count_rounded <- round(avg_crime_count)

    # Debug: print the average crime count to verify
    print(paste("Average crime count for missing LSOA", missing_lsoa, ":", avg_crime_count))

    # Check if avg_crime_count is not calculated properly
    if(is.na(avg_crime_count) || avg_crime_count == 0) {
      # Apply a different strategy or set a default value other than 0 if that makes sense in your context
      avg_crime_count <- 1 # Example: setting a default value if mean calculation fails
    }

    # Update the dataset with the calculated average for the missing LSOA
    asb_grouped_by_lsoa <- rbind(asb_grouped_by_lsoa, data.frame(lsoa_code = missing_lsoa, count = avg_crime_count_rounded))
}

# Left_join the now cleaned data back into the shapefile
asb_lsoa <- left_join(shp_file, asb_grouped_by_lsoa, by = c("lsoa21cd" = "lsoa_code"))


#### Using GGPPLOT2

This ggplot code snippet creates a heat map visualizing the distribution of ASB incidents across LSOAs. The use of the magma color palette from viridis ensures that the visualization is both attractive and perceptually accurate, enabling viewers to easily see where ASB incidents are more or less concentrated. 


In [ ]:
ggplot(data = asb_lsoa) +
  geom_sf(aes(fill = count), color = NA) + # geom_sf plots the spatial data, fill controls the color based on ASB count
  scale_fill_viridis_c(option = "magma", direction = -1, na.value = "white", 
                       guide = guide_colorbar(title = "ASB Count")) + # Adjusts the color scale
  labs(title = "ASB Incidents by LSOA", 
       subtitle = "Visualised using ggplot2") + 
  theme_minimal()


#### Using leaflet


This code snippet creates an interactive map using the leaflet package in R to visualize Anti-Social Behaviour (ASB) incidents across Lower Super Output Areas (LSOAs) based on the asb_lsoa dataset. The dataset is expected to be an sf object containing geometries for each LSOA and a count variable representing the number of ASB incidents in each area


In [ ]:
# Define a color palette function based on the 'count' column
colorPalette <- colorNumeric(palette = "viridis", domain = asb_lsoa$count, na.color = "transparent")

# Create the Leaflet map
leaflet(asb_lsoa) %>%
  addProviderTiles(providers$CartoDB.Positron) %>% # Adds a light-themed base map
  addPolygons(
    fillColor = ~colorPalette(count), # Apply the color based on the 'count' column
    weight = 1, # Border weight
    opacity = 1, # Border opacity
    color = "white", # Border color
    fillOpacity = 0.7, # Fill opacity
    popup = ~paste0("LSOA: ", lsoa21cd, "<br>Count: ", count) # Popup content
  ) %>%
  addLegend("bottomright", pal = colorPalette, values = ~count,
            title = "ASB Counts",
            opacity = 1)

asb_lsoa <- st_transform(asb_lsoa, 4326)


### Time Series Analysis 

After completing our spatial interpolation, we now shift our focus to analyzing trends over time for Anti-Social Behaviour (ASB) incidents across Surrey Heath.

Crime patterns are rarely static, and time series analysis helps us:
✔ Identify trends – Are ASB incidents increasing or decreasing?
✔ Detect seasonal patterns – Do incidents peak in certain months?
✔ Compare variations across LSOAs – Are some areas more volatile than others?

What We Will Do
In this section, we will:
1️⃣ Load and process 12 months of ASB data dynamically.
2️⃣ Visualize ASB trends per LSOA (messy but insightful).
3️⃣ Aggregate ASB trends across all LSOAs for a clearer picture.
4️⃣ Apply a rolling average to smooth fluctuations and highlight long-term trends.

By the end of this section, we will have a comprehensive understanding of how ASB incidents evolve over time and how we can use temporal patterns to inform crime prevention strategies.


### Load & Process ASB Crime Data  
We dynamically load **12 months of crime data** from the `Data/crime_data` directory.


In [ ]:
# Define path to the crime data folder
#crime_data_path <- "Data/crime_data"
# crime_data_path <- "/Users/user/Documents/Mapping_Crime_Data_R_2025/Data/crime_data"
# 
# # List all monthly folders dynamically (last 12 months)
# monthly_folders <- dir_ls(crime_data_path, type = "directory") %>% tail(12)
# 
# # Function to read each month's crime data
# read_crime_data <- function(folder) {
#   file_path <- dir_ls(folder, regexp = "surrey-street.csv$")
#   read_csv(file_path) %>%
#     clean_names() %>%
#     mutate(month = basename(folder)) # Extract folder name as new column
# }
# 
# # Read all crime data from the last 12 months
# all_crime_data <- map_dfr(monthly_folders, read_crime_data)

##
library(tidyverse)
library(fs)
library(janitor)

# Define base GitHub URL
github_base <- "https://raw.githubusercontent.com/UKDataServiceOpen/Mapping_Crime_Data_R_2025/main/Data/crime_data"

# List last 12 months (update as needed)
months <- format(seq(Sys.Date(), by = "-1 month", length.out = 12), "%Y-%m")

# Function to download and read each month's crime data
read_crime_data <- function(month) {
  file_url <- paste0(github_base, "/", month, "/", month, "-surrey-street.csv")
  
  # Temporary file to store downloaded data
  temp_file <- tempfile(fileext = ".csv")
  
  # Download the file
  download.file(file_url, temp_file, mode = "wb")
  
  # Read the CSV file
  read_csv(temp_file) %>%
    clean_names() %>%
    mutate(month = month) # Add month as a column
}

# Read and merge all crime data from the last 12 months
all_crime_data <- map_dfr(months, possibly(read_crime_data, NULL))

# View dataset
head(all_crime_data)


---

### Filter & Aggregate ASB Crimes  
We filter for **Anti-Social Behaviour** incidents in **Surrey Heath** and aggregate by **LSOA & Month**.


In [ ]:
# Filter for ASB crimes in Surrey Heath
asb_crime_data <- all_crime_data %>%
  filter(crime_type == "Anti-social behaviour") %>%
  filter(str_detect(lsoa_name, "^Surrey Heath"))

# Aggregate by LSOA and Month
asb_grouped_by_lsoa <- asb_crime_data %>%
  mutate(year = substr(month, 1, 4), month = substr(month, 6, 7)) %>%
  group_by(lsoa_code, year, month) %>%
  summarise(count = n(), .groups = "drop") %>%
  arrange(lsoa_code, year, month)

# Create a date column for time series plotting
asb_grouped_by_lsoa <- asb_grouped_by_lsoa %>%
  mutate(date = as.Date(paste(year, month, "01", sep = "-")))


---
 
### **1. Individual LSOA ASB Trends** (*Messy but insightful*)  
Each line represents a different LSOA.


In [ ]:
# Plot individual LSOA trends
ggplot(asb_grouped_by_lsoa, aes(x = date, y = count, group = lsoa_code)) +
  geom_line(alpha = 0.3, color = "gray") +  # Faint individual lines
  geom_smooth(method = "loess", se = FALSE, color = "blue", size = 1.2) +  # Trend line
  labs(title = "ASB Trends Over Time (Per LSOA)",
       x = "Month", y = "Number of ASB Incidents") +
  theme_minimal()


---

### **2. Aggregated ASB Trends** (*Cleaner and more readable*)  
We sum ASB counts across **all LSOAs** per month.


In [ ]:
# Aggregate ASB count by month
asb_trend <- asb_grouped_by_lsoa %>%
  group_by(date) %>%
  summarise(total_count = sum(count), .groups = "drop")

# Plot aggregated trend
ggplot(asb_trend, aes(x = date, y = total_count)) +
  geom_line(color = "blue", size = 1.2) +  # Trend line
  geom_point(color = "red", size = 2) +  # Highlight monthly points
  labs(title = "Aggregated ASB Trends Over Time",
       x = "Month", y = "Total ASB Incidents") +
  theme_minimal()


---

### **3. Smoothed ASB Trend (Rolling Average)**  
We apply a **3-month rolling average** to smooth fluctuations.


In [ ]:
# Add 3-month rolling average
asb_trend <- asb_trend %>%
  mutate(rolling_avg = rollmean(total_count, k = 3, fill = NA, align = "right"))

# Plot rolling average trend
ggplot(asb_trend, aes(x = date)) +
  geom_line(aes(y = total_count), color = "lightblue", size = 1) +  # Raw data
  geom_line(aes(y = rolling_avg), color = "blue", size = 1.5) +  # Smoothed trend
  geom_point(aes(y = total_count), color = "red", size = 2) +
  labs(title = "Aggregated ASB Trends with Rolling Average",
       x = "Month", y = "Total ASB Incidents") +
  theme_minimal()


#### Optional Activity 

Compare the heat maps now using the ASB data WITH the missing values. Follow the steps below by first preparing your dataset and then making the plots. 

*1) Filter for just ASB within Surrey Heath 


In [ ]:
asb_data2 <- crime %>%
  filter(.... == "Anti-social behaviour") %>% 
  filter(str_detect(...., "^Surrey Heath"))


*2) Group by LSOA and summarise the crime count



In [ ]:
asb_grouped_by_lsoa2 <- asb_data %>%
  group_by(.....) %>%
  summarise(.....=n())


*3) Left_join the new grouped ASB dataset to the shapefile

Remember we are skipping the imputation method here as we want to see the affect of including missing values into our maps. So go ahead and simply join the grouped data to the shapefile


In [ ]:
asb_lsoa_missing <- left_join(...., ...., by = c("lsoa21cd" = "lsoa_code"))



*4) Now plot with ggplot 



In [ ]:
ggplot(data = ...) +
  geom_sf(aes(fill = ...elt()), color = NA) + 
  scale_fill_viridis_c(option = "magma", direction = -1, na.value = "white", 
                       guide = guide_colorbar(title = "ASB Count")) + # Adjusts the color scale
  labs(title = "ASB Incidents by LSOA", 
       subtitle = "Visualised using ggplot2") + 
  theme_minimal()


*5) Now plot with leaflet 



In [ ]:
# Define a color palette function based on the 'count' column
colorPalette <- colorNumeric(palette = "viridis", domain = asb_lsoa_missing$count, na.color = "transparent")

# remember to first transform the object into the correct CRS
asb_lsoa_missing <- .....(asb_lsoa_missing, ....)

# Create the Leaflet map
leaflet(....) %>%
  addProviderTiles(providers$CartoDB.Positron) %>% # Adds a light-themed base map
  addPolygons(
    fillColor = ~colorPalette(count), 
    weight = 1, # Border weight
    opacity = 1, # Border opacity
    color = "white", # Border color
    fillOpacity = 0.7, # Fill opacity
    popup = ~paste0("LSOA: ", lsoa21cd, "<br>Count: ", count) 
  ) %>%
  addLegend("bottomright", pal = colorPalette, values = ~count,
            title = "ASB Counts",
            opacity = 1)


--------------------------------------------------------------------------------------------------------------------------

## 📚 ADDITIONAL TOPICS 📚 


## Load packages 


In [ ]:
knitr::opts_chunk$set(echo = TRUE)

# for data reading/manipulation 
library(dplyr)
library(tidyr)
library(readr)
library(tibble)
library(janitor)
library(readxl)
# for spatial data and gis
library(sf)
library(ggmap)
library(ggplot2)
library(ggspatial)
library(ggspatial)
library(spdep)
library(leaflet) 
library(RColorBrewer)
library(tmap)
library(geojsonio)


## Explore the data using ggmap and ggplot (Google API)

### Setting up a an API 

Setting up a connetion to Google's API;

1. First: set up an account with Google.

2. Second: to obtain an API go to [https://cloud.google.com/maps-platform/] and follow the onscreen instructions. This documentation shows you how to input the requisite information (e.g. your API key) into R, and it also shows you a few tools that can help you work with the credentialing. In summary the onscreen instructions are as follows. 
 - click 'get started' 
 - click the credentials tab on the left hand side of the screen 
 - click 'create credentials at the top of the screen (visible by the '+' symbol)
 - Select 'API key'
 - Select copy and now you have your personal API key 
 
 
 *You will be asked to set up a billing account but you will not need to pay to follow along with the below steps. Google Cloud has a limited amount of free storage (200 dollars free a month) so remember to always check your storage use and the usage cost under the 'cost breakdown' panel.*
 

3. To authorise the API you have to enable some of Google's API which are required for the project. This includes 
- Google Maps JavaScript API
- Google Maps Geocoding API
- Google Maps Geolocation API
- Google Maps Places API
- Google Maps Static API

If you do not authorise the above API you will recieve and error message as such "Geocoding Service: This API project is not authorised to use the API". To authorise said APIs go to 
a. https://console.cloud.google.com/apis/dashboard?project=missing-314119
b. Click '+ Enable APIS and SERVICES' at the top of the screen
c. Search for each API listed above and click the 'Enable button' 

Once all APIs are enabled, wait 5 minutes for changes to take place, and then move on to the next step to register your Google API to ggmaps. 


4. Third; you need to tell ggmap about your API by using the 'register_google()' function. It would look something like *register_google(key = "hahfuibfiu324898249dbhsgag")* (this is a fake key). This will then set your API key for the current session, Every time you restart R you will have to request a new API key or you can set it permanently by adding 'write = TRUE', the full code being....

*register_google(key = "[your key]", write = TRUE)*

For more information on how to register your decive to the Google API please visit [https://rdrr.io/cran/ggmap/man/register_google.html]. It is incredibly important to not that each API is private and should not be shared with anyone. Users should also be aware that ggmap has no mechanism with which to safeguard the private key once registered with R. 

The basic idea driving ggmap is to take a downloaded map image, plot it as a context layer using ggplot2, and then plot additional content layers of data, statistics, or models on top of the map. In ggmap, downloading a map as an image and formatting the image for plotting is done with the get_map function. More specifically, the get_map is a wrapper function

It is important to note that when using ggmap, users have to first setup an account with Google, enable the relevant API, and then tell R about the user's setup. Do not worry about doing this for this workshop as we will demonstrate the code below. If you would like to run this in your own time then please refer to the steps above which details how to obtain your API and how to enable the services. 


### For one specific area


In [ ]:
# ## Ariel Map of Surrey
qmplot(longitude, latitude, data = crime, colour = crime_type, size = I(3), darken = .3)
# 

# ## Lets just say you were interested in a specific area (in this example we will use Crawley 002B)
# 
# ## Ariel Map of Crawley 002B
 geocode("Crawley")
# 

Crawley <- c(long = -0.152210, lat = 51.15813)
map <- get_map(Crawley, zoom = 13, scale = 1)
ggmap(map)
# 
ggmap(map) +
  geom_point(aes(longitude, latitude), data = crime) 
# 
# 
# ## Colour the Crime Type
# 

ggmap(map) +
   geom_point(aes(longitude, latitude, colour = crime_type), data = crime) 

# 

 ggmap(map) +
   geom_point(aes(longitude, latitude, size = crime_type, colour = crime_type), data = crime) 


### For one specific crime type 



In [ ]:
#subset for just ASB
subset <- crime[which(crime$crime_type == "Anti-social behaviour"),]

#Create the crawley map again 
map <- ggmap(get_googlemap(center = c(long = -0.152210, lat = 51.15813), 
                           zoom = 10, 
                           maptype = "terrain", 
                           color = "color"))
#print the map
print(map)


#overlay the crime data 
map <- ggmap(get_googlemap(center = c(long = -0.631027, lat = 51.215485), 
                           zoom = 9, 
                           maptype = "terrain", 
                           color = "color")) +
  geom_point(data = subset, aes(x = longitude, y = latitude), 
             colour = "red", size = .5)

#print 
print(map)


#lets make a density map using ggplot
map <- ggmap(get_googlemap(center = c(long = -0.152210, lat = 51.15813), 
                           zoom = 9, 
                           maptype = "terrain", 
                           color = "color")) +
  stat_density2d(aes(x = longitude, y = latitude, fill = ..level..), 
               alpha = .2, 
               bins = 10, data = subset, geom = "polygon")


print(map)


### Binning data

Binning, can be thought of as a two-dimensional histogram (shading of the bins take the heights of the bars). You first need to convert the sf data.frame geometry column into a data.frame with separate x, y columns. 

How do you separate the coordinates? 

Luckily a function to do this already exists [https://github.com/r-spatial/sf/issues/231]. The below code is converting a sfc_point to seperate x, y columns 


In [ ]:
sfc_as_cols <- function(x, names = c("x","y")) {
  stopifnot(inherits(x,"sf") && inherits(sf::st_geometry(x),"sfc_POINT"))
  ret <- sf::st_coordinates(x)
  ret <- tibble::as_tibble(ret)
  stopifnot(length(names) == raster::ncol(ret))
  x <- x[ , !names(x) %in% names]
  ret <- setNames(ret,names)
  dplyr::bind_cols(x,ret)
}

sf_seperate <- sfc_as_cols(sf, c("lng", "lat")) 


ggplot(sf_seperate, aes(lng, lat)) +   
  annotation_map_tile() +
  stat_binhex(bins = 30) +                                           
  scale_fill_gradientn(colours = c("white","red"), name = "Frequency")   


#hexagonal = stat_binhex() 
#rectangle = stat_bin2d()
#heat = stat_density2d()  


### Interactive Maps; Leaflet

Leaflet is one of the most popular open-source JavaScript libraries for interactive maps. For more information you can view this link here [https://rstudio.github.io/leaflet/]


In [ ]:
## Subsetting for just ASB 
asb <- subset(crime, crime_type == "Anti-social behaviour")

m <- leaflet(data = asb) %>%
  addProviderTiles("Stamen.Toner") %>% 
  addMarkers(lng=~longitude, lat=~latitude, popup=~as.character(location), label = ~as.character(location))
m


### Other imporant functions 

- Jittering: 

Jittering indeed means just adding random noise to a vector of numeric values, by default this is done in jitter-function by drawing samples from the uniform distribution. The range of values in the jittering is chosen according to the data, if amount-parameter is not provided. It helps to grasp where the density of observations is high. 

There are a few packages that offer this method including the 'geom_jitter' function found in the ggplot2 package [https://ggplot2.tidyverse.org/reference/geom_jitter.htm]. Additionally, the 'rjitter' function under the spatsat package. The function rjitter is generic, with methods for point patterns (described here) and for some other types of geometrical objects. Each of the points in the point pattern X is subjected to an independent random displacement. More information can be found here; [https://rdrr.io/cran/spatstat.geom/man/rjitter.html]

-  st_intersect():

This function is also under the sf package and is used to intersect two objects between two sets of objects. More information can be found here [https://r-spatial.github.io/sf/reference/geos_binary_ops.htm]



## 🎉 THANK YOU FOR PARTICIPATING! 🎉

We appreciate your time and effort in completing the **Crime Mapping in R Workshop**.  
We hope you found the hands-on exercises and analyses insightful and useful for your research or projects.

If you have any questions, feedback, or would like to stay connected, feel free to reach out! 📩  


### 📩 CONTACT DETAILS

**Workshop Instructor:** Nadia Kennar  
📧 **Email:** [nadia.kennar@manchester.ac.uk]  
🌐 **GitHub Repository:** [Crime Mapping in R - GitHub](https://github.com/UKDataServiceOpen/Mapping_Crime_Data_R_2025)  
 

### REFERENCES

Below are the key references and data sources used throughout this workshop:

- **UK Crime Data Source**: [Police API & Open Data](https://data.police.uk/)  
- **UK Census Data**: [Office for National Statistics (ONS)](https://www.ons.gov.uk/)  
- **Spatial Analysis in R**: [Simple Features for R (`sf` package)](https://r-spatial.github.io/sf/articles/sf1.html)  
- **Crime Mapping Techniques**: [GIS & Crime Mapping Research Paper](https://www.researchgate.net/publication/CrimeMapping)  
- **Leaflet for R**: [Interactive Mapping Documentation](https://rstudio.github.io/leaflet/)  

---

### Want to Keep Learning?
If you're interested in further developing your GIS, spatial analysis, and R skills, here are some additional resources:  

- 📖 **Book:** "Geocomputation with R" - [Link](https://geocompr.robinlovelace.net/)  
- 🎥 **Videos & Tutorials:** [R Spatial Analysis YouTube Series](https://www.youtube.com/watch?v=xyz123)  
- 💡 **Online Courses:** Coursera GIS & R Programming for Spatial Data  


###  **NEXT STEPS**
- Feel free to explore the **additional topics** section for more advanced spatial analysis techniques.  
- If you want to apply what you’ve learned, try **analyzing your own crime or census dataset**!  
- Let us know what you think—your feedback helps improve future workshops!  

👋 **Thank you once again, and happy mapping!** 🗺️
